In [3]:
import os 
import pandas as pd
import evaluate
import spacy


In [4]:
model_prediction_path = "/people/gerald/Documents/repositories/Educational-French-Question-Answering/predictions/predictions-mbart-alignement"

In [5]:
predictions = {}
for fname in os.listdir(model_prediction_path):
    predictions[fname.split('.')[0].split("-")[1] + "-" + fname.split('.')[0].split("-")[-1]] =\
        pd.read_csv(os.path.join(model_prediction_path, fname))

In [6]:
bertscore = evaluate.load('bertscore', model = "xlm-roberta-large")
rouge  = evaluate.load('rouge')
nlp = spacy.load("fr_core_news_lg")

In [7]:
predictions['piaf-default_processing']

,qid,qtype,default_selection,generated_text,ground_truth_text
0,0,NONE,True,Combien sont les Spartiates par rapport à leur...,Quel était la proportion de Spartiates par rap...
1,1,NONE,True,Pour qui les Spartiates sont-ils maltraités?,Qui haissent les Spartiates?
2,2,NONE,True,Pourquoi la politique des Lacédémoniens à l'ég...,Comment est dictée la politique des Lacédémoni...
3,3,NONE,True,Comment les Égaux se déplacent-ils?,Quelle est la tradition chez les Spartiates?
4,4,NONE,True,Que craignent les Spartiates?,Que craignaient les les Spartiates?
...,...,...,...,...,...
762,762,NONE,True,Qu'est-ce qui ne dément pas l'aspect brutal?,Quelle est la singularité des orques?
763,763,NONE,True,Que les orques apprécient-ils?,De quoi se nourrissent les orques?
764,764,NONE,True,Qu'est ce que les orques ne dédaignent pas?,"En cas de disette, de quoi sont capables les o..."
765,765,NONE,True,De quoi les orques sont-ils souvent composés?,"malgré leur cruauté, quel rang militaire occup..."


In [12]:
total_dictionary = {}
for k, df_pred in predictions.items():
    dataset_name, seed_method = k.split("-")
    if dataset_name not in total_dictionary:
        total_dictionary[dataset_name] = {}

    prediction = [p for p in df_pred["generated_text"]]
    references = [r for r in df_pred["ground_truth_text"]]
    score = rouge.compute(predictions=prediction, rouge_types=["rougeL"], references=references, use_aggregator=False, tokenizer= lambda x : [t.text for t in nlp.tokenizer(x)])
    df_pred['rougeL'] = score['rougeL']
    score = bertscore.compute(predictions=prediction, references=references, model_type="xlm-roberta-large")
    df_pred['f1'] = score['f1']

    mean_group = df_pred.groupby("qid").mean()['rougeL']
    mean_group = df_pred.groupby("qid").mean()['rougeL']
    max_group = df_pred.groupby("qid").max()['rougeL']
    min_group = df_pred.groupby("qid").min()['rougeL']
    count_group = df_pred.groupby("qid").count()['rougeL']
    
    rmegm = round(mean_group.mean() * 100, 1)
    rmegs = round(mean_group.std() * 100, 1)
    rmagm = round(max_group.mean() * 100, 1)
    rmags = round(max_group.std() * 100, 1 )
    rmigm = round(min_group.mean() * 100, 1 )
    rmigs = round(min_group.std() * 100, 1 )

    mean_group = df_pred.groupby("qid").mean()['f1']
    mean_group = df_pred.groupby("qid").mean()['f1']
    max_group = df_pred.groupby("qid").max()['f1']
    min_group = df_pred.groupby("qid").min()['f1']
    count_group = df_pred.groupby("qid").count()['f1']
  
    bmegm = round(mean_group.mean() * 100, 1)
    bmegs = round(mean_group.std() * 100, 1)
    bmagm = round(max_group.mean() * 100, 1)
    bmags = round(max_group.std() * 100, 1 )
    bmigm = round(min_group.mean() * 100, 1 )
    bmigs = round(min_group.std() * 100, 1 )
    
    print("\t%s:"%(k))
    res = "$%s$ & $%s$ & $%s$ & $%s$ & $%s$ & $%s$ & $%s$"%(rmegm,  bmegm,  rmagm,  bmagm,  rmigm,  bmigm, round(count_group.mean(),1))
    total_dictionary[dataset_name][seed_method] = res


	piaf-default_processing:
	isahit-nouns_phrase:
	freelance-default_processing:
	freelance-nouns_phrase:
	freelance-keyphrase:
	freelance-cobject:
	isahit-default_processing:
	fquad-cobject:
	fquad-default_processing:
	piaf-entities:
	fquad-entities:
	piaf-cobject:
	fquad-nouns_phrase:
	freelance-entities:
	piaf-keyphrase:
	isahit-entities:
	fquad-keyphrase:
	isahit-cobject:
	isahit-keyphrase:
	piaf-nouns_phrase:


In [20]:
def formating_dataset(data,
                      dataset_order=["fquad", "piaf", "freelance", "isahit"],
                      seed_order=[("default_processing", "DEF"),("entities", "ENT"),
                                  ("nouns_phrase", "NP"), ("keyphrase", "KP"), ('cobject', 'OBJ')]):
    for d in dataset_order:
        print("\mediumhline")
        print("\multirow{%s}{*}{%s}"%(len(seed_order), d))
        for i, s in enumerate(seed_order):
            print("&%s&%s\\\\"%(s[1], data[d][s[0]]))
            

In [21]:
formating_dataset(total_dictionary)

\mediumhline
\multirow{5}{*}{fquad}
&DEF&$41.8$ & $90.9$ & $41.8$ & $90.9$ & $41.8$ & $90.9$ & $1.0$\\
&ENT&$30.9$ & $89.1$ & $36.8$ & $90.2$ & $26.1$ & $88.1$ & $2.3$\\
&NP&$27.7$ & $88.7$ & $43.6$ & $91.3$ & $15.7$ & $86.2$ & $7.1$\\
&KP&$29.6$ & $89.0$ & $36.5$ & $90.3$ & $22.8$ & $87.7$ & $2.0$\\
&OBJ&$32.3$ & $89.3$ & $35.7$ & $90.0$ & $29.2$ & $88.7$ & $1.7$\\
\mediumhline
\multirow{5}{*}{piaf}
&DEF&$38.5$ & $90.5$ & $38.5$ & $90.5$ & $38.5$ & $90.5$ & $1.0$\\
&ENT&$29.7$ & $88.9$ & $35.2$ & $89.9$ & $25.3$ & $88.0$ & $2.4$\\
&NP&$26.0$ & $88.3$ & $39.8$ & $90.8$ & $15.5$ & $86.0$ & $6.4$\\
&KP&$26.9$ & $88.5$ & $33.4$ & $89.6$ & $20.5$ & $87.3$ & $2.0$\\
&OBJ&$30.6$ & $89.1$ & $33.4$ & $89.6$ & $28.2$ & $88.6$ & $1.6$\\
\mediumhline
\multirow{5}{*}{freelance}
&DEF&$27.5$ & $89.2$ & $27.5$ & $89.2$ & $27.5$ & $89.2$ & $1.0$\\
&ENT&$23.9$ & $88.5$ & $28.2$ & $89.3$ & $20.2$ & $87.7$ & $2.5$\\
&NP&$23.2$ & $88.3$ & $36.6$ & $90.6$ & $13.1$ & $86.0$ & $8.5$\\
&KP&$24.4$ & $88.6$ & $

In [61]:
tr_key = {
    'knowledge' : 'course',
    'document_study': 'raisoning'
}
qtype_dictionary = {}
for k, df_pred in predictions.items():
    dataset_name, seed_method = k.split("-")
    if dataset_name not in qtype_dictionary:
        qtype_dictionary[dataset_name] = {}
    gmean = df_pred.groupby('qid').mean()
    gmax = df_pred.groupby('qid').max()
    gmin = df_pred.groupby('qid').min()
    count = df_pred.groupby(['qid' ]).count()

    gmean['qtype'] = gmax["qtype"]
    gmin['qtype'] = gmax["qtype"]
    gmax['qtype'] = gmax["qtype"]
    count['qtype'] = gmax["qtype"]

    rmean = gmean.groupby('qtype').mean()['rougeL']
    fmean = gmean.groupby('qtype').mean()['f1']
    rmax = gmax.groupby('qtype').mean()['rougeL']
    fmax = gmax.groupby('qtype').mean()['f1']
    rmin = gmin.groupby('qtype').mean()['rougeL']
    fmin = gmin.groupby('qtype').mean()['f1']
    acount = count.groupby(['qtype' ]).mean()['f1']
    
    dico = {}
    for key in rmax.to_dict().keys():
        nkey = key
        if key in tr_key:
            nkey = tr_key[key]
        dico[nkey]  = {}
        dico[nkey]["rmean"] = rmean[key]
        dico[nkey]["fmean"] = fmean[key]
        dico[nkey]["rmax"] = rmax[key]
        dico[nkey]["fmax"] = fmax[key]
        dico[nkey]["rmin"] = rmin[key]
        dico[nkey]["fmin"] = fmin[key]
        dico[nkey]["count"] = acount[key]

    qtype_dictionary[dataset_name][seed_method] = dico

In [66]:
def qtype_dictionary_to_table(data,
                              dataset_order=["freelance", "isahit"],
                              seed_to_use="default_processing",
                              qtype_order=[('factual', 'FACT'), ('vocabulary', 'VOCA'),
                                           ('course', "COUR"), ("raisoning", "SYNT")
                                          ]):
    for d in dataset_order:
        print("\mediumhline")
        print("\multirow{%s}{*}{%s}"%(len(qtype_order), d))
        #print(data[d][seed_to_use])
        for i, s in enumerate(qtype_order):
            ### ONLY WITH RECENT PYTHON VERSION ELSE ORDER IN DICTIONARY ARE NOT PRESERVED
            print("&%s&%s\\\\"%(s[1], "&".join([str(round(rs * 100, 1)) if(rs < 1) else str(round(rs, 1)) for rs in data[d][seed_to_use][s[0]].values()])))
            

In [67]:

qtype_dictionary_to_table(qtype_dictionary)

\mediumhline
\multirow{4}{*}{freelance}
&FACT&28.2&89.5&28.2&89.5&28.2&89.5&1.0\\
&VOCA&48.0&91.4&48.0&91.4&48.0&91.4&1.0\\
&COUR&28.3&89.3&28.3&89.3&28.3&89.3&1.0\\
&SYNT&22.2&88.5&22.2&88.5&22.2&88.5&1.0\\
\mediumhline
\multirow{4}{*}{isahit}
&FACT&50.0&92.6&50.0&92.6&50.0&92.6&1.0\\
&VOCA&37.8&90.7&37.8&90.7&37.8&90.7&1.0\\
&COUR&36.2&90.5&36.5&90.6&35.8&90.5&1.1\\
&SYNT&21.7&88.2&28.3&89.4&16.1&87.0&2.4\\


In [100]:
import numpy as np
gmean = filtered_df.groupby('id').mean()
gmax = filtered_df.groupby('id').max()
gmin = filtered_df.groupby('id').min()
count = filtered_df.groupby(['id' ]).count()

gmean['question_type'] = question_type_list
gmin['question_type'] = question_type_list
gmax['question_type'] = question_type_list
count['question_type'] = question_type_list

rmean = gmean.groupby('question_type').mean()['rougeL']
fmean = gmean.groupby('question_type').mean()['f1']
rmax = gmax.groupby('question_type').mean()['rougeL']
fmax = gmax.groupby('question_type').mean()['f1']
rmin = gmin.groupby('question_type').mean()['rougeL']
fmin = gmin.groupby('question_type').mean()['f1']
acount = count.groupby(['question_type' ]).mean()['f1']



for key_qt in np.unique(question_type_list):
    print("\n"+key_qt+" : ")
    print("$%s$ & $%s$ & $%s$ & $%s$ & $%s$ & $%s$ & $%s$"%(
        round(rmean[key_qt]*100, 2 ) , round(fmean[key_qt]*100, 2 ) , round(rmax[key_qt]*100, 2 ) , 
        round(fmax[key_qt]*100, 2 ) , round(rmin[key_qt]*100, 2 ) , round(fmin[key_qt]*100, 2 ) , round(acount[key_qt], 2 ) )) #,count))
print("$%s$ & $%s$ & $%s$ & $%s$ & $%s$ & $%s$ & $%s$"%(round(gmean.mean()['rougeL'] *100,2),
                                                        round(gmean.mean()['f1'] *100,2),
    round(gmax.mean()['rougeL']*100, 2 ), round(gmax.mean()['f1']*100, 2 ),
    round(gmin.mean()['rougeL']*100, 2 ),round(gmin.mean()['f1']*100, 2 ),
    round(count.mean()['f1'], 2)))



NONE : 
$27.79$ & $88.4$ & $46.21$ & $91.64$ & $12.53$ & $85.18$ & $12.37$
$27.79$ & $88.4$ & $46.21$ & $91.64$ & $12.53$ & $85.18$ & $12.37$


/tmp/ipykernel_30331/3423135992.py:27: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print("$%s$ & $%s$ & $%s$ & $%s$ & $%s$ & $%s$ & $%s$"%(round(gmean.mean()['rougeL'] *100,2),
/tmp/ipykernel_30331/3423135992.py:28: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  round(gmean.mean()['f1'] *100,2),
/tmp/ipykernel_30331/3423135992.py:29: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  round(gmax.mean()['rougeL']*100, 2 ), round(gmax.mean()['f1']*100, 2 ),
/tmp/ipykernel_30331/3423135992.py:

In [101]:
gmean.groupby('question_type').min()['rougeL']

question_type
NONE    0.0
Name: rougeL, dtype: float64

In [63]:
fmin['document_study']

0.8043004870414734